## Running the seq2seq Training Algorithm Pipeline

## 1 Input Pre-Processing

#### 1.1 Creating a dictionary & generating embeddings
This process is handled entirely via the lexemes module. Many downstream tasks will require using either embeddings or the dictionary from them, so we'll start with that.

In [ ]:
from seq2seq.lexemes import *

#c5-3 is updated local2 in classic
df_train = pd.read_csv('PATH/TO/FILE.csv')

#Using vec model 1, building encoder embeddings
lex = lexemes(with_nonce=True, lemmatization=True)
f2id = lex.id_dic(df_all, ['tref', 'lex', 'head', 'lex.dep', 'lex.head'])
f_vecs, f_err = lex.embeds(f2id)

#Creating target values (from cmsource) and decoder embeddings (t_vecs)
tar = lexemes(with_nonce=True, lemmatization=True)
t2id = tar.id_dic(df_all, ['head', 'tref', 'tref.dep'])
t_vecs, t_err = tar.embeds(t2id, dimensions=300)

#### 1.2 Converting text to indeces using lexeme dictionaries
We have our dictionaries, so let's now pre-process our input text. The files listed above are organized such that decoder inputs are in rows in certain columns, and encoder inputs are organized vertically in certain columns. This was merely the easiest way to keep allthe data organized at the time, but for future work the inputs module below can handle items in rows or columns as needed.

In [ ]:
from seq2seq.inputs import *
encoder, decoder = inputs(f2id), inputs(t2id)
encX, decX, Y = encoder.bulk_vertical(df_all, columns=['lex']), decoder.bulk_horizontal(df_all,columns=['head', 'tref.dep', 'tref']), decoder.bulk_decoderY(df_all,columns=['head', 'tref.dep', 'tref'])

## 2 Model Generation
The seq2seq model module is kerasNN. You can swap it out for any other file you'd like. The .py merely needs to contain the model you're looking for, and in fact one could circumvent it entirely and just write their own code, on the fly, and then pass it to the training module. It matters not.

In [ ]:
from seq2seq.kerasNN import *
#Start by initializing a model object.
m = net(trainable_embeds=True, embeddings_list=[[f2id, 300, f_vecs],[t2id, 300, t_vecs]], decoder_initial_step=nonce, rnn_units=300)

#The kerasNN module has two pre-loaded seq2seq models-- the base (b5), and experimental (e1)
#The number of inputs will dictate the number of inputs required in pre-processing.
m.e1(inputs=['lex'])

## 3 Training
Training is handled via the training module. To use it, initialize it with the model you're going to be training, the encoder data, the decoder data, and the target values built in inputs. A representations model is not required, but can be added later via train.reps = /MODEL/

In [ ]:
from seq2seq.modelHub import *
gym = hub(m.nn1, encX, decX, Y, m.nn_rep)
gym.train(epochs=20, cutoff=.9

If you have validation data, add it in the .train function as a list or tuple, tagged to parameter "validation_data", organized as (validation_encoder_data, validataion_decoder_data, validation_target_values)

A representations model is not required, but can be added later via train.reps = /MODEL/